# Assignment: Data Wrangling
## `! git clone https://github.com/DS3001/wrangling`
## Do Q2, and one of Q1 or Q3.

**Q1.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about? 
  > From the abstract, this paper is about data tidying, an important component of data cleaning.

  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  > A standard would make the intial data cleaning easier as one doesn't have to start from scratch. So a tidy data standard is to help make data analysis easier and more efficient by establishing a standard way to organize data values in a dataset. 

  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."

  > The first sentence means that a tidy datatset follow a consistent structure, making them similar to each other. However, a messy dataset can have a variety of inconsistencies, making each one unique in its own way.
  > The second sentence means that it is easy to identify observations (rows) and variables (column) in a specific dataset. However, defining these concepts all around if tricky.

  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  > Values: numbers (quantitative) or strings (qualitative). Values are organized in two ways. Every value belongs to a variable, which contains all values that measure the same underlying attribute across units (ex. height, temperature, money), and observations, which contains all values measured on the same unit (ex. person, day, race).

  5. How is "Tidy Data" defined in section 2.3?
  - Each variable forms a column
  - Each observation forms a row
  - Each type of observational unit forms a table

  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
 > The 5 most common problems with a messy dataset are:
  - Column headers are values, not variable names
  - Multiple variables are stored in one column
  - Variables are stored in both rows and columns
  - Multiple types of observational units are stored in the same table
  - A single observational unit is stored in multiple tables.
> The data in Table 4 is messy because the incomes are used as column headers instead of being values in a single column. Should be 'religion', a new column called'income', and 'frequency.' Melting a dataset is essentially turning column values into rows. 

  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?
  > Table 11 is messy because there is column for each possible day in the month, some days/months do not have value and is left blank. However table 12 melts all the days into a single variable 'date'. The missing values are dropped to conserve space and each row represents the meterological measurements for a single day.

  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?
  > The “chicken-and-egg” problem with tidy data is about whether to clean and organize data before analysis or to analyze messy data and tidy it as needed. Hadley Wickham hopes future work will create better tools and practices for efficient data tidying, making it easier for data scientists to work with data seamlessly across different programming languages.

**Q2.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the `./data/sharks.csv` data covered in the lecture, clean the "Type" variable as well as you can, and explain the choices you make.
3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [5]:
import pandas as pd


#1
airbnb_data = pd.read_csv('./data/airbnb_hw.csv')
#print(airbnb_data['Price'].head())
airbnb_data['Price'] = airbnb_data['Price'].replace('[\,]', '', regex=True).astype(float)
missing_values = airbnb_data['Price'].isnull().sum()
print(f"Number of missing values in Price: {missing_values}")

# Choices made: removed commas to ensure the values can be converted to numeric and converted
# the Price column to a float type

Number of missing values in Price: 0


In [15]:
import numpy as np

df = pd.read_csv('./data/sharks.csv', low_memory=False)


print("Initial value counts:")
print(df['Type'].value_counts())

df['Type'] = df['Type'].replace(
    ['Sea Disaster', 'Boat', 'Boating', 'Boatomg'], 'Watercraft'
).replace(
    ['Invalid', 'Questionable', 'Unconfirmed', 'Unverified', 'Under investigation'], np.nan
)

print("Cleaned value counts:")
print(df['Type'].value_counts())

# Choices made: standarizes similar categories into single term "Watercraft"
#Replaced invalid or questionable entries with np.nan to handle missing values

Initial value counts:
Type
Unprovoked             4716
Provoked                593
Invalid                 552
Sea Disaster            239
Watercraft              142
Boat                    109
Boating                  92
Questionable             10
Unconfirmed               1
Unverified                1
Under investigation       1
Boatomg                   1
Name: count, dtype: int64
Cleaned value counts:
Type
Unprovoked    4716
Provoked       593
Watercraft     583
Name: count, dtype: int64


In [34]:

df = pd.read_parquet('./data/justice_data.parquet')
release = df['WhetherDefendantWasReleasedPretrial']
# replace missing values with np.nan
release = release.replace(9,np.nan) 
print(release.value_counts(),'\n')
sum(release.isnull()) 
df['WhetherDefendantWasReleasedPretrial'] = release
del release


WhetherDefendantWasReleasedPretrial
1.0    19154
0.0     3801
Name: count, dtype: int64 



In [35]:
imposed = df['ImposedSentenceAllChargeInContactEvent'] 
type = df['SentenceTypeAllChargesAtConvictionInContactEvent']

imposed = pd.to_numeric(imposed, errors='coerce')
sentence_na = imposed.isnull()
print(np.sum(sentence_na),'\n') 

print(pd.crosstab(sentence_na, type),'\n')
imposed = imposed.mask(type==4, 0)
imposed = imposed.mask(type==9, np.nan)

sentence_na = imposed.isnull()
print(pd.crosstab(sentence_na, type), '\n')
print(np.sum(sentence_na),'\n')

df['ImposedSentenceAllChargeInContactEvent'] = imposed
del imposed, type

9053 

SentenceTypeAllChargesAtConvictionInContactEvent     0     1    2     4    9
ImposedSentenceAllChargeInContactEvent                                      
False                                             8720  4299  914     0    0
True                                                 0     0    0  8779  274 

SentenceTypeAllChargesAtConvictionInContactEvent     0     1    2     4    9
ImposedSentenceAllChargeInContactEvent                                      
False                                             8720  4299  914  8779    0
True                                                 0     0    0     0  274 

274 



**Q3.** Many important datasets contain a race variable, typically limited to a handful of values often including Black, White, Asian, Latino, and Indigenous. This question looks at data gathering efforts on this variable by the U.S. Federal government.

1. How did the most recent US Census gather data on race?
2. Why do we gather these data? What role do these kinds of data play in politics and society? Why does data quality matter?
3. Please provide a constructive criticism of how the Census was conducted: What was done well? What do you think was missing? How should future large scale surveys be adjusted to best reflect the diversity of the population? Could some of the Census' good practices be adopted more widely to gather richer and more useful data?
4. How did the Census gather data on sex and gender? Please provide a similar constructive criticism of their practices.
5. When it comes to cleaning data, what concerns do you have about protected characteristics like sex, gender, sexual identity, or race? What challenges can you imagine arising when there are missing values? What good or bad practices might people adopt, and why?
6. Suppose someone invented an algorithm to impute values for protected characteristics like race, gender, sex, or sexuality. What kinds of concerns would you have?